# Fit and export the sleep staging classifier

In [ ]:
import os
import glob
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
sns.set(font_scale=1.25)

# Define path
wdir = "/Volumes/JAWA/SHERLOCK/features/"
wdir_demo = "/Volumes/JAWA/SHERLOCK/demo/"

## Load the feature files

**Method 1: Loop across all datasets**

In [ ]:
# feat_files = glob.glob(wdir + "features_nsrr*.parquet")

# df = []
# for f in tqdm(feat_files):
#     # Load current file
#     print(f)
#     tmp = pd.read_parquet(f)
#     # Convert dtypes and downcast float
#     tmp['age'] = tmp['age'].astype('int8')
#     tmp['male'] = tmp['male'].astype('category')
#     cols_float = tmp.select_dtypes(np.float64).columns.tolist()
#     tmp[cols_float] = tmp[cols_float].astype(np.float32)
#     # Append to main dataframe and delete tmp
#     df.append(tmp)
#     del tmp
    
# df = pd.concat(df)
# print("There are %i unique nights" % df.index.get_level_values(0).nunique())
# df.head().round(2)

In [ ]:
# Save the concatenated file (slow!)
# df.to_parquet(wdir + "features_all.parquet")

**Method 2: Concatenated file**

In [ ]:
# Or we can simply use a pre-saved concatenated file
df = pd.read_parquet(wdir + "features_all.parquet")

# print("There are %i unique nights" % df.index.get_level_values(0).nunique())
# df.head().round(2)

In [ ]:
# Check units of datasets
df.groupby('dataset')['eeg_iqr'].describe().round(2)

### Add demographics (age, sex, race, BMI)

In [ ]:
# First, let's merge with the main demographics
df_demo = pd.concat([pd.read_csv(f) for f in glob.glob(wdir_demo + "*")])
df_demo['subj'] = df_demo['subj'].astype(str)
# Fix an invalid BMI ("A") in MrOs
df_demo['bmi'] = df_demo['bmi'].apply(lambda x: x if isinstance(x, (int, float)) else np.nan)
df_demo.drop(columns=['overall', 'visit', 'gender', 'visitnumber', 'male', 'age'], inplace=True)
df_demo

In [ ]:
grp_subj = df.groupby(level=0, as_index=True)[['age', 'male', 'dataset']].first()
grp_subj.reset_index(inplace=True)
# Preprocessing before merge
grp_subj['subj'] = grp_subj['subj'].astype(str)
grp_subj['dataset'] = grp_subj['dataset'].str.upper()
grp_subj['dataset'] = grp_subj['dataset'].replace({'SHHS1': 'SHHS'})
# Merge
grp_subj = grp_subj.merge(df_demo, how="left")
grp_subj = grp_subj.sort_values(by=['dataset', 'subj']).reset_index(drop=True)
grp_subj

In [ ]:
# Missing values
grp_subj.isna().sum()

In [ ]:
# Number of nights per dataset
grp_subj['dataset'].value_counts(sort=False)

In [ ]:
# Number of hours / epochs
df.shape[0] / 120

In [ ]:
grp_subj['male'].value_counts(normalize=True)

In [ ]:
# Plot age distribution
def mean_std(x):
    print(f"{x.mean():.2f} ± {x.std():.2f} (min = {x.min():.2f}, median = {x.median()}, max = {x.max():.2f})")

grp_subj['age'].agg(mean_std)
grp_subj['age'].hist()
plt.xlabel("Age");

In [ ]:
grp_subj['ahi'].agg(mean_std)

In [ ]:
grp_subj['bmi'].agg(mean_std)

In [ ]:
100 * grp_subj['ethnicity'].value_counts(normalize=True).round(3)

### Create different combinations of predictors

In [ ]:
# EEG also includes the time columns
cols_all = df.columns
cols_time = cols_all[cols_all.str.startswith('time_')].tolist()
cols_eeg = cols_all[cols_all.str.startswith('eeg_')].tolist() + cols_time
cols_eog = cols_all[cols_all.str.startswith('eog_')].tolist()
cols_emg = cols_all[cols_all.str.startswith('emg_')].tolist()
cols_demo = ['age', 'male']

In [ ]:
# Define predictors
X_eeg = df[cols_eeg]
X_eeg_eog = df[cols_eeg + cols_eog]
X_eeg_eog_emg = df[cols_eeg + cols_eog + cols_emg]
X_eeg_eog_emg_demo = df[cols_eeg + cols_eog + cols_emg + cols_demo]

X_all = [X_eeg, X_eeg_eog, X_eeg_eog_emg, X_eeg_eog_emg_demo]
X_names = ['eeg', 'eeg+eog', 'eeg+eog+emg', 'eeg+eog+emg+demo']

# Define target and groups
y = df['stage']
subjects = df.index.get_level_values(0).to_numpy()

In [ ]:
# % of each sleep stage
y.value_counts(normalize=True).plot.barh(xlabel="Stage", ylabel="Proportion");

In [ ]:
# Define hyper-parameters
params = dict(
    boosting_type='gbdt',
    n_estimators=200,
    max_depth=7,
    num_leaves=30,
    colsample_bytree=0.8,
    importance_type='gain',
    n_jobs=1
)

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# compute_class_weight('balanced', np.unique(y), y)

# Manually define class weight
# class_weight = None
# class_weight = "balanced"
class_weight = "custom"

if class_weight == "custom":
    # Every epoch of N1 counts for 1.8
    params['class_weight'] = {'N1': 1.8, 'N2': 1, 'N3': 1, 'R': 1, 'W': 1}
else:
    params['class_weight'] = class_weight

****

## Fit the training set and export the trained classifier

In [ ]:
# Parallel processing when building the trees.
params['n_jobs'] = 6

# Loop across combs of predictors
for X, name in zip(tqdm(X_all), X_names):
    # Fit
    clf = LGBMClassifier(**params)
    clf.fit(X, y)

    # Print the accuracy on the training dataset: shouldn't be too high..!
    print("%s (%i features) - training accuracy: %.3f" % 
        (name, X.shape[1], clf.score(X, y)))
    
    # Export trained classifier
    outdir = "/Volumes/JAWA/SHERLOCK/classifiers/"
    if params['class_weight'] is not None:
        fname = outdir + 'clf_%s_lgb_%s_%s.joblib' % \
        (name, params['boosting_type'], class_weight)
    else:
        fname = outdir + 'clf_%s_lgb_%s.joblib' % \
        (name, params['boosting_type'])
    joblib.dump(clf, fname, compress=True)

    # Also save directly to YASA
    # outdir_yasa = "/Users/raphael/GitHub/yasa/yasa/classifiers/"
    # fname_yasa = outdir_yasa + 'clf_%s_lgb.joblib' % name
    # joblib.dump(clf, fname_yasa, compress=True)

    # Export feature importance
    df_imp = pd.Series(clf.feature_importances_, index=clf.feature_name_, 
                        name='Importance').round()
    df_imp.sort_values(ascending=False, inplace=True)
    df_imp.index.name = 'Features'
    df_imp.to_csv(fname[:-7] + ".csv")